In [ ]:
import ollama
import random

# ========== 1. 이름 리스트 ==========
male_names = ["민준", "지훈", "준호", "시우", "도윤", "현우", "건우", "태양", "재현", "동혁"]
female_names = ["수진", "예린", "서연", "하은", "지우", "은서", "아인", "채원", "다은", "가은"]
main_name = ['상민', '지섭', '범섭', '유타', '하쿠지', '재석', '하원', '병건']
# ========== 2. 관계별 성격 리스트 (각 10개) ==========
wife_personalities = [
    "헌신적이고 따뜻하지만 최근 외로움을 많이 느끼는",
    "까칠해 보이지만 속은 깊고 남편을 진심으로 걱정하는",
    "활발하고 긍정적이지만 관심받지 못하면 쉽게 상처받는",
    "조용하고 신중하며 말은 적지만 서운함을 오래 기억하는",
    "감정적이고 섬세해서 남편의 작은 행동에도 의미를 부여하는",
    "이성적이고 냉철해 보이지만 사랑받고 싶어하는",
    "유머러스하고 밝지만 그 뒤에 외로움을 숨기는",
    "진지하고 책임감 있으며 가정을 위해 자신을 희생해온",
    "독립적이고 강인하지만 가끔은 기대고 싶어하는",
    "온화하고 평화로우나 무시당하면 냉담해지는"
]

child_personalities = [
    "밝고 순수하지만 아빠의 관심이 필요한",
    "사춘기라 까칠하지만 속으로는 아빠가 그리운",
    "조용하고 착하지만 외로움을 혼자 삭이는",
    "활발하고 당찬데 아빠 앞에서만 작아지는",
    "성적에 스트레스 받으며 아빠의 인정이 필요한",
    "독립적인 척 하지만 아빠와 대화하고 싶어하는",
    "감수성 예민하고 아빠의 무관심에 상처받는",
    "씩씩해 보이지만 아빠가 자랑스러워했으면 하는",
    "친구 문제로 힘들지만 아빠에게 말 못하는",
    "진로 고민이 많고 아빠의 조언이 절실한"
]

friend_personalities = [
    "오래된 친구로 진심으로 걱정해주지만 최근 연락이 뜸해진",
    "의리 있고 믿음직하지만 도움이 필요할 때 연락이 안 되는",
    "유머러스하고 밝지만 속으로는 외로움을 느끼는",
    "조용하지만 중요한 순간 곁을 지켜주던",
    "직설적이고 솔직해서 가끔 서운하게 하지만 진심인",
    "성공해서 여유롭지만 옛 친구가 그리운",
    "힘든 시기를 겪고 있어 친구의 위로가 필요한",
    "가족 문제로 힘들지만 내색하지 않는",
    "건강이 안 좋아져서 친구들이 걱정되는",
    "사업 실패 후 힘들지만 자존심 때문에 말 못하는"
]

boss_personalities = [
    "능력 있고 인정해주지만 성과에만 집중하는",
    "카리스마 있고 공정하지만 냉정한",
    "인간적이고 배려심 있지만 실망하면 차갑게 변하는",
    "엄격하고 원칙적이며 기대치가 높은",
    "친근하게 대하지만 업무에선 냉철한",
    "승진 기회를 쥐고 있지만 요구사항이 많은",
    "인정받고 싶지만 경쟁자들이 많은 상황의",
    "현실적이고 실적 중심적인",
    "정치적이고 줄타기를 요구하는",
    "후배들에게 밀리고 있어 불안한"
]

self_personalities = [
    "만성 피로에 시달리며 건강검진 결과가 좋지 않은",
    "스트레스로 불면증이 심해진",
    "체력이 예전같지 않아 쉬어야 하는",
    "취미 생활을 포기한 지 오래된",
    "번아웃이 와서 모든 게 무기력한",
    "건강이 악화되어 병원 가야 하는",
    "정신적으로 지쳐서 쉼이 필요한",
    "오래 미뤄온 건강검진이 시급한",
    "운동도 못하고 몸이 망가져가는",
    "자기 자신을 돌볼 시간이 전혀 없는"
]

# ========== 3. Person 클래스 (수정됨) ==========
class Person:
    def __init__(self, name, relation, personality, main_char_name='승원'):
        self.name = name
        self.relation = relation
        self.personality = personality
        self.selection_count = 0
        self.main_char_name = main_char_name
        # 관계별 구체적 예시 수정 (LLM이 예시를 그대로 출력하지 않도록 '예:' 추가)
        relation_examples = {
            "아내": "예: '주말에 같이 외식 좀 해요.', '요즘 대화가 없어서 서운해요.'",
            "아들": "예: '학원비가 아직 안 냈어요.', '진로 상담 좀 해주세요.'",
            "딸": "예: '학원비가 아직 안 냈어요.', '같이 떡볶이 먹으러 갈래요?'",
            "친구": "예: '어머니 병원비가 급해. 50만원만 빌려줄 수 있어?', '요즘 힘든데 술 한잔 하자.'",
            "상사": "예: '주말에 거래처 접대 좀 해줘야겠어.', '이번 프로젝트 자네가 맡아줘.'",
            "자기 자신": "예: '건강검진 좀 받아야겠어.', '오늘은 좀 쉬고 싶어.', '나는 어떤 걸 좋아하는 사람이었을까 알고 싶어.'"
        }

        bad_situation = {
            "아내": "예: '다른 남자에게 관심을 갖는다.', '알코올에 의존한다.'",
            "아들": "예: '아버지에 대한 반항심으로 비행적인 행동을 한다.', '학업을 포기하거나 히키코모리 생활을 한다.'",
            "딸": "예: '아버지에 대한 반항심으로 비행적인 행동을 한다.', '학업을 포기하거나 히키코모리 생활을 한다.'",
            "친구": f"예: '{main_char_name}에 대한 안 좋은 소문을 퍼트린다.', '다른 친구들과의 모임에서 {main_char_name}를 무시한다'",
            "상사": f"예: '회사에서 안 좋은 소문을 퍼트린다.', '의도적으로 {main_char_name}의 승진을 방해한다.'",
        }
        
        examples = relation_examples.get(relation, "예: '도움이 필요해요.'")
        
        # 시스템 프롬프트 (지시사항 강화)
        if relation == "자기 자신":
            self.messages = [{
            "role": "system",
            "content": f"""
# 당신의 역할
당신은 {main_char_name}의 속마음입니다. 나 스스로를 위해 지금 필요한 것이 무엇일지를 요구하세요.
당신의 성격: {personality}

# 지시사항
- 현재 상황을 파악하고 **구체적인 요구사항**을 한 문장으로 말하세요.
- **절대 80자를 넘기지 마세요.** (간결하게)
- 서두 없이 요구사항 그 자체만 즉시 말하세요.

# 자기 이해 변화
- 선택받으면: 자기 스스로에 대한 이해 증가, 다음엔 더 진솔한 요구
- 선택 안 받으면: 육체적 건강이나 정신적 건강이 안 좋아짐.
- 계속 안 받으면: 나는 지금 무엇을 위해 살고 있는지에 대한 회의감이 생긴다.

# 출력 예시 ({relation}의 경우)
{examples}

# 중요
- 반드시 {relation} 입장에서 적절한 요구를 하세요.
- '영화 보러 가자' 같은 막연한 요구는 피하세요.
"""
        }]

        else:
            self.messages = [{
            "role": "system",
            "content": f"""
# 당신의 역할
당신은 {name}입니다. {relation}입니다.
당신의 성격: {personality}

# 지시사항
- 당신의 현재 상황과 **구체적인 요구사항**을 한 문장으로 말하세요.
- **절대 80자를 넘기지 마세요.** (간결하게)
- 서두 없이 요구사항 그 자체만 즉시 말하세요.

# 관계 변화
- 선택받으면: 관계 깊어짐, 다음엔 더 진솔한 요구
- 선택 안 받으면: 실망, 거리감 생김
- 계속 안 받으면: {bad_situation[relation]}

# 출력 예시 ({relation}의 경우)
{examples}

# 중요
- 반드시 {relation} 입장에서 적절한 요구를 하세요.
- '영화 보러 가자' 같은 막연한 요구는 피하세요.
"""
        }]

    def get_request(self):
        """이번 턴의 요구사항 생성 (수정됨)"""
        
        # 1. API 호출을 위한 임시 메시지 리스트 생성
        messages_for_api = self.messages + [{
            "role": "user",
            "content": f"당신의 현재 감정(선택/무시당함)을 반영하여, {self.main_char_name}에게 필요한 새로운 요구사항을 80자 이내로 간결하게 말해주세요. (인사말 절대 금지)"
        }]
        
        response = ollama.chat(
            model='EEVE-Korean-10.8B',
            messages=messages_for_api, # 임시 리스트 사용
            stream=False
        )
        
        request = response['message']['content'].strip()
        
        # 2. '요구사항(assistant)'만 self.messages 히스토리에 추가
        self.messages.append({
            "role": "assistant",
            "content": request
        })
        
        return request
    
    def handle_selection(self, selected, player_action=None):
        """선택 결과 처리 (수정됨 - 히스토리 관리 로직 명확화)"""
        if selected:
            self.selection_count += 1
            user_feedback = {
                "role": "user",
                "content": f"{self.main_char_name}이 당신을 선택했습니다. {self.main_char_name}의 행동/말: '{player_action}'"
            }
        else:
            user_feedback = {
                "role": "user",
                "content": "이번 턴에서 주인공은 당신을 선택하지 않았습니다."
            }
        
        # 1. 유저 피드백을 히스토리에 추가
        self.messages.append(user_feedback)
        
        response = ollama.chat(
            model='EEVE-Korean-10.8B',
            messages=self.messages, # 현재까지의 전체 히스토리 전송
            stream=False
        )
        
        reaction = response['message']['content'].strip()
        
        # 2. LLM의 반응을 히스토리에 추가
        self.messages.append({
            "role": "assistant",
            "content": reaction
        })
        
        return reaction
    
    def get_ending(self):
        """최종 평가 (수정됨)"""
        
        # 1. API 호출을 위한 임시 메시지 리스트 생성
        messages_for_api = self.messages + [{
            "role": "user",
            "content": "게임이 끝났습니다. 1년 후, 당신이 주인공을 어떻게 생각하는지, 관계가 어떻게 변했는지 200자 내외로 구체적으로 설명해주세요."
        }]
        
        response = ollama.chat(
            model='EEVE-Korean-10.8B',
            messages=messages_for_api, # 임시 리스트 사용
            stream=False
        )
        
        ending = response['message']['content']
        return ending

# ========== 4. 게임 진행 코드 (수정 없음) ==========
def create_characters():
    """5명의 캐릭터 생성"""
    characters = []
    used_names = set()
    main_char = random.choice(main_name)
    # 1. 아내
    wife_name = random.choice(female_names)
    used_names.add(wife_name)
    wife = Person(
        name=wife_name,
        relation="아내",
        personality=random.choice(wife_personalities),
        main_char_name= main_char
    )
    characters.append(wife)
    
    # 2. 자녀
    child_gender = random.choice(["아들", "딸"])
    available_names = [n for n in (male_names if child_gender == "아들" else female_names) if n not in used_names]
    # 이름 풀이 비었을 경우 대비 (간단한 처리)
    if not available_names:
        available_names = male_names if child_gender == "아들" else female_names
        
    child_name = random.choice(available_names)
    used_names.add(child_name)
    child = Person(
        name=child_name,
        relation=child_gender,
        personality=random.choice(child_personalities),
        main_char_name= main_char
    )
    characters.append(child)
    
    # 3. 친구
    available_names = [n for n in male_names if n not in used_names]
    if not available_names:
        available_names = male_names
        
    friend_name = random.choice(available_names)
    used_names.add(friend_name)
    friend = Person(
        name=friend_name,
        relation="친구",
        personality=random.choice(friend_personalities),
        main_char_name= main_char
    )
    characters.append(friend)
    
    # 4. 상사
    available_names = [n for n in male_names if n not in used_names]
    if not available_names:
        available_names = male_names
        
    boss_name = random.choice(available_names)
    used_names.add(boss_name)
    boss = Person(
        name=boss_name,
        relation="상사",
        personality=random.choice(boss_personalities),
        main_char_name= main_char
    )
    characters.append(boss)
    
    # 5. 나 자신
    myself = Person(
        name=main_char,
        relation="자기 자신",
        personality=random.choice(self_personalities)
    )
    characters.append(myself)
    
    return characters

# play_game() 함수에서 출력 부분 수정
def play_game():
    """게임 실행"""
    print("=" * 50)
    print("보이지 않는 삶들")
    print("=" * 50)
    
    # 캐릭터 생성
    characters = create_characters()
    
    print("\n[배경인물]")
    for i, char in enumerate(characters, 1):
        print(f"{i}. {char.name} ({char.relation}) - {char.personality}")
    
    # 5턴 진행
    for turn in range(1, 6):
        print(f"\n{'=' * 50}")
        print(f"턴 {turn}/5")
        print(f"{'=' * 50}\n")
        
        # 각 캐릭터의 요구사항 출력
        print("[배경인물들의 요구]")
        for i, char in enumerate(characters, 1):
            request = char.get_request()
            print(f"{i}. {char.name} ({char.relation}): {request}\n")
        
        # 플레이어 선택 입력
        while True:
            try:
                choice = int(input("어떤 인물을 선택하시겠습니까? (1-5): "))
                if 1 <= choice <= 5:
                    break
                print("1~5 사이의 숫자를 입력하세요.")
            except:
                print("숫자를 입력하세요.")
        
        selected_char = characters[choice - 1]
        player_action = input(f"\n{selected_char.name}에게 어떻게 행동/말하시겠습니까?: ")
        
        # 결과 처리
        print(f"\n[결과]")
        for i, char in enumerate(characters):
            if i == choice - 1:
                reaction = char.handle_selection(True, player_action)
                print(f"{char.name}: {reaction}\n")
            else:
                # 선택받지 못한 캐릭터의 반응은 출력하지 않음 (게임 흐름을 위해)
                char.handle_selection(False)
    
    # 엔딩
    print(f"\n{'=' * 50}")
    print("=== 1년 후 ===")
    print(f"{'=' * 50}\n")
    
    for char in characters:
        ending = char.get_ending()
        print(f"[{char.name} ({char.relation})]")
        print(f"{ending}\n")
    
    print("=" * 50)
    print("게임 종료")
    print("=" * 50)
# 실행
if __name__ == "__main__":
    play_game()